<p align="center">
  <img src="https://i.ytimg.com/vi/Wm8ftqDZUVk/maxresdefault.jpg" alt="FIUBA" width="25%"/>
  </p>
  
# **Trabajo Práctico 2: Críticas Cinematográficas**
### **Grupo**: 11 - Los Pandas 🐼
### **Cuatrimestre**: 2ºC 2023
### **Corrector**: Mateo Suster
### **Integrantes**:
- ### 106861 - Labollita, Francisco
- ### 102312 - Mundani Vegega, Ezequiel
- ###  97263 - Otegui, Matías Iñaki

# Modelo XGBoost

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import xgboost as xgb

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, make_scorer
from sklearn.model_selection import StratifiedKFold, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report

In [6]:
reviews = pd.read_csv('train.csv')

## Implementación del bag of words

In [7]:
vectorizerTotal = CountVectorizer(strip_accents='unicode', dtype='uint16')
vectorizerTotal.fit_transform(reviews['review_es'])

# Primeros 20 elementos
print(vectorizerTotal.get_feature_names_out()[:20])
# Elementos del medio
print(vectorizerTotal.get_feature_names_out()[10000:10020])
# Últimos 20 elementos
print(vectorizerTotal.get_feature_names_out()[-20:])

['00' '000' '00000' '00000000000' '0000000000001' '00000001' '00001'
 '0001' '00015' '000dm' '001' '002' '003830' '006' '0069' '007' '0079'
 '007the' '0080' '0083']
['antisocial' 'antisociales' 'antiste' 'antisunciados' 'antit'
 'antitabaco' 'antitanque' 'antiterroristas' 'antitesis' 'antitetico'
 'antithesis' 'antithetical' 'antitica' 'antitm' 'antitreideros'
 'antitrust' 'antivirus' 'antiwar' 'antm' 'antoina']
['zyuranger' 'zz' 'zzzz' 'zzzzip' 'zzzzz' 'zzzzzzzzz' 'zzzzzzzzzzzzzz'
 'zzzzzzzzzzzzzzzzz' 'zzzzzzzzzzzzzzzzzzzz' 'zzzzzzzzzzzzzzzzzzzzz'
 'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz'
 'zzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzzz' 'æbler' 'æon' 'æsthetic'
 'østbye' 'þo' 'þorleifsson' 'יגאל' 'כרמון']


Se ve que varias "palabras" serán números, algunas tendrán símbolos no pertenecientes al alfabeto español y también se comprueba que están palabras españolas.

## Feature engineering del bag of words

En primer lugar, siendo que todas las palabras que inician una oración empiezan en mayúscula, se hará que todas las letras de palabras con una sola mayúscula sean transformadas a minúsculas. De tal manera que en el siguiente ejemplo, las dos variaciones de hermosa sean una misma palabra: "Hermosa película" y "Esta película es hermosa".

In [8]:
matrizApariciones = vectorizerTotal.fit_transform(reviews['review_es'])

In [9]:
matrizSiAparece = matrizApariciones.toarray()
matrizApariciones = matrizApariciones.toarray()

In [10]:
matrizSiAparece[matrizSiAparece > 0] = 1

In [11]:
words_df = pd.DataFrame()
words_df['Palabra'] = vectorizerTotal.get_feature_names_out()
words_df['Apariciones Totales'] = matrizApariciones.sum(axis=0).tolist() #Cuántas veces aparece la palabra
words_df['Apariciones'] = matrizSiAparece.sum(axis=0).tolist()           #En cuántas reviews aparece la palabra

In [12]:
#Se cuentan en cuántas reviews positivas aparece cada palabra
listaAparicionesPositivas = np.zeros(shape=len(matrizSiAparece[0])).astype('int32')
for i in range(reviews.shape[0]):
    if (reviews.iloc[i]['sentimiento'] == 'positivo'):
        listaAparicionesPositivas += matrizSiAparece[i]

#Se cuentan en cuántas reviews negativas aparece cada palabra
listaAparicionesNegativas = np.zeros(shape=len(matrizSiAparece[0])).astype('int32')
for i in range(reviews.shape[0]):
    if (reviews.iloc[i]['sentimiento'] == 'negativo'):
        listaAparicionesNegativas += matrizSiAparece[i]

In [13]:
words_df['Apariciones positivas'] = listaAparicionesPositivas
words_df['Apariciones negativas'] = listaAparicionesNegativas
words_df['Fracción apariciones positivas'] = words_df['Apariciones positivas'] / words_df['Apariciones']
words_df['Fracción apariciones negativas'] = words_df['Apariciones negativas'] / words_df['Apariciones']
words_df['Tasa de positividad'] = (words_df['Apariciones positivas'] - words_df['Apariciones negativas']) / words_df['Apariciones']
words_df.sort_values(by='Apariciones', inplace=True, ascending=False)
words_df.head(10)

,Palabra,Apariciones Totales,Apariciones,Apariciones positivas,Apariciones negativas,Fracción apariciones positivas,Fracción apariciones negativas,Tasa de positividad
41364,de,661907,47992,23949,24043,0.499021,0.500979,-0.001959
128119,que,395365,47245,23501,23744,0.497428,0.502572,-0.005143
93125,la,405160,47147,23496,23651,0.498356,0.501644,-0.003288
55382,en,276429,45938,22882,23056,0.498106,0.501894,-0.003788
53511,el,253915,45037,22381,22656,0.496947,0.503053,-0.006106
162711,una,170883,43530,21815,21715,0.501149,0.498851,0.002297
58627,es,183244,43210,21708,21502,0.502384,0.497616,0.004767
162707,un,186195,43041,21390,21651,0.496968,0.503032,-0.006064
111087,no,145805,42253,19918,22335,0.471398,0.528602,-0.057203
60252,esta,119728,40952,20067,20885,0.490013,0.509987,-0.019975


## Se entrena un modelo básico de XGBoost

In [24]:
# Se transforman los positivos en 1 y los negativos en 0
# Transformar los valores de reviews['sentimiento'] a 1 y 0
reviews['sentimiento'] = reviews['sentimiento'].apply(lambda x: 1 if x == 'positivo' else 0)

# Dividir los datos en conjuntos de entrenamiento y prueba
x_train, x_test, y_train, y_test = train_test_split(
    reviews['review_es'], reviews['sentimiento'], test_size=0.2, random_state=42)

# Crear una matriz de términos de documento utilizando CountVectorizer
vectorizer = CountVectorizer()
x_train_counts = vectorizer.fit_transform(x_train)

# Entrenar el modelo de Random Forest
model = xgb.XGBClassifier(random_state=0)

In [25]:
model.fit(x_train_counts, y_train)

# Transformar los datos de prueba y hacer predicciones
x_test_counts = vectorizer.transform(x_test)
y_pred = model.predict(x_test_counts)

# Imprimir el informe de clasificación
print(classification_report(y_test, y_pred))

c:\Users\ezequ\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\data.py:520: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(data):


              precision    recall  f1-score   support

           0       0.86      0.83      0.84      4961
           1       0.84      0.86      0.85      5039

    accuracy                           0.84     10000
   macro avg       0.85      0.84      0.84     10000
weighted avg       0.85      0.84      0.84     10000



## Predicción del conjunto test

In [27]:
test_df = pd.read_csv('test.csv')

#Transformo todas las letras a minúscula
test_df['review_es'] = test_df['review_es'].str.lower()

x_test_counts = vectorizer.transform(test_df['review_es'])
y_pred_test = model.predict(x_test_counts)

test_df['sentimiento'] = y_pred_test
test_df['sentimiento'] = test_df['sentimiento'].apply(lambda x: 'positivo' if x == 1 else 'negativo')

test_df.drop("review_es", axis=1, inplace=True)
test_df.to_csv('sample_solution.csv', index=False)